In [1]:
# Donchian Weekly Classic Trend Following System
import gta_indicator
import gta_prices
import gta_signals
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'WOOD'
timeframe = 'Weekly'
filename = f'{exchange}_{tidm}_prices.csv'
p1 = 48 # System 1 look back period.
p2 = 24 # System 2 look back period.
p3 = 12 # System 3 look back period.
p4 = 6  # System 4 look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
charges = 11.95  # Commission per trade.
sduty = 0.0  # Stamp Duty percentage.

In [3]:
# Function definitions.
def shares_div4(shares):
    '''Modify number of shares to be purchased to be divisible by 4.'''
    s = pd.Series(shares.values)
    for i in s.index:
        while s.iloc[i] % 4 != 0:
            s.iloc[i] += 1
    return shares

In [4]:
# Import weekly closing prices.
prices = gta_prices.weekly(exchange, tidm)

In [5]:
# Donchian channels.
dc1 = gta_indicator.donchian(prices, p1)
dc2 = gta_indicator.donchian(prices, p2)
dc3 = gta_indicator.donchian(prices, p3)
dc4 = gta_indicator.donchian(prices, p4)

In [6]:
# System 1 entries & exits.
s1 = pd.concat([prices, dc1], axis=1)
s1['sys'] = 1
s1['buy'] = np.where(s1.close > s1.upr, 1, 0)
s1['sell'] = np.where(s1.close < s1.mid, 1, 0)
s1['state'] = gta_signals.state_signal(s1.buy, s1.sell, p1)
s1['entry'] = np.where(np.logical_and(s1.state == 1, s1.state.shift(periods=1) == 0), 1, 0)
s1['exit'] = np.where(np.logical_and(s1.state == 0, s1.state.shift(periods=1) == 1), 1, 0)

In [7]:
# System 2 entries & exits.
s2 = pd.concat([prices, dc2], axis=1)
s2['sys'] = 2
s2['buy'] = s1.entry
s2['sell'] = np.where(s2.close < s2.mid, 1, 0)
s2['state'] = gta_signals.state_signal(s2.buy, s2.sell, p2)
s2['entry'] = np.where(np.logical_and(s2.state == 1, s2.state.shift(periods=1) == 0), 1, 0)
s2['exit'] = np.where(np.logical_and(s2.state == 0, s2.state.shift(periods=1) == 1), 1, 0)

In [8]:
# System 3 entries & exits.
s3 = pd.concat([prices, dc3], axis=1)
s3['sys'] = 3
s3['buy'] = s1.entry
s3['sell'] = np.where(s3.close < s3.mid, 1, 0)
s3['state'] = gta_signals.state_signal(s3.buy, s3.sell, p3)
s3['entry'] = np.where(np.logical_and(s3.state == 1, s3.state.shift(periods=1) == 0), 1, 0)
s3['exit'] = np.where(np.logical_and(s3.state == 0, s3.state.shift(periods=1) == 1), 1, 0)

In [9]:
# System 4 entries & exits.
s4 = pd.concat([prices, dc4], axis=1)
s4['sys'] = 4
s4['buy'] = s1.entry
s4['sell'] = np.where(s4.close < s4.mid, 1, 0)
s4['state'] = gta_signals.state_signal(s4.buy, s4.sell, p4)
s4['entry'] = np.where(np.logical_and(s4.state == 1, s4.state.shift(periods=1) == 0), 1, 0)
s4['exit'] = np.where(np.logical_and(s4.state == 0, s4.state.shift(periods=1) == 1), 1, 0)

In [10]:
# All entries & exits.
td = pd.concat([s1[s1.entry == 1] , s1[s1.exit == 1], s2[s2.exit == 1], s3[s3.exit == 1], s4[s4.exit == 1]], axis=0)
td = td.sort_index()

In [11]:
# Position size (buy).
td['volatility'] = np.where(td.entry == 1, abs((td.mid - td.close) / td.close).round(3), 0)
td['risk_amt'] = np.where(td.entry == 1, ((position_size * risk_pct) / td.volatility).round(2), 0)
td['shares'] = np.where(td.entry == 1, (td.risk_amt / td.close).astype('int'), 0)
td.shares = shares_div4(td.shares) # Modify number of shares to be purchased to be divisible by 4.
td.risk_amt = np.where(td.entry == 1, (td.close * td.shares).round(2), 0) # Adjust risk amount for revised share count.

# Position size (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        shares = row['shares']
    elif row['exit'] == 1:
        td.at[index, 'shares'] = int(shares / 4)

In [12]:
# Charges & stamp duty.
td['charges'] = charges
td['sduty'] = np.where(td.entry==1, ((sduty / 100) * td.close * td.shares).round(2), 0)

# Cost (buy).
td['cost'] = np.where(td.entry == 1, ((td.close * td.shares) + td.charges + td.sduty).round(2), 0)

# Cost (sell).
for index, row in td.iterrows():
    if row['entry'] == 1:
        cost = row['cost']
    elif row['exit'] == 1:
        td.at[index, 'cost'] = (cost / 4).round(2)

# Value (sell).
td['value'] = np.where(td.exit == 1, ((td.close * td.shares) - td.charges).round(2), 0)

# Profit.
td['profit'] = np.where(td.exit == 1, td.value - td.cost, 0)

# Cumulative profit.
td['cum_profit'] = td.profit.cumsum()

# Percentage return.
td['pct'] = np.where(td.exit == 1, ((td.profit / td.cost) * 100).round(1), 0)

# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    elif row['exit'] == 1:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1) * 100).round(1)

In [13]:
# Reindex by trade.
td['trade'] = td.state.cumsum()
td = td.reset_index()
td = td.set_index('trade')
td

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,cum_profit,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2009-09-18,8.4567,8.9045,8.4567,8.9045,8.7387,5.3119,7.02530,1,1,0,1,1,0,0.211,7123.60,800,11.95,0.0,7135.55,0.00,0.00,0.00,0.0,0,0.0
1,2009-10-30,9.0700,9.0700,8.2205,8.2205,9.3151,8.4567,8.88590,4,0,1,0,0,1,0.000,0.00,200,11.95,0.0,1783.89,1632.15,-151.74,-151.74,-8.5,42,-53.8
1,2009-10-30,9.0700,9.0700,8.2205,8.2205,9.3151,7.9027,8.60890,3,0,1,0,0,1,0.000,0.00,200,11.95,0.0,1783.89,1632.15,-151.74,-303.48,-8.5,42,-53.8
1,2010-06-04,10.5070,10.6280,10.3600,10.3730,11.6830,9.2100,10.44650,2,0,1,0,0,1,0.000,0.00,200,11.95,0.0,1783.89,2062.65,278.76,-24.72,15.6,259,22.7
1,2011-05-20,11.2200,11.2500,11.0600,11.0800,12.7000,9.6211,11.16055,1,0,1,0,0,1,0.000,0.00,200,11.95,0.0,1783.89,2204.05,420.16,395.44,23.6,609,13.5
2,2013-01-04,10.4100,10.9000,10.3250,10.9000,10.5000,8.3147,9.40735,1,1,0,1,1,0,0.137,10943.60,1004,11.95,0.0,10955.55,0.00,0.00,395.44,0.0,0,0.0
2,2013-04-05,12.1800,12.3900,11.7600,11.7950,12.3300,11.4750,11.90250,4,0,1,0,0,1,0.000,0.00,251,11.95,0.0,2738.89,2948.60,209.71,605.15,7.7,91,34.4
2,2013-05-31,12.2200,12.2200,11.6600,11.6950,12.6000,11.4700,12.03500,3,0,1,0,0,1,0.000,0.00,251,11.95,0.0,2738.89,2923.50,184.61,789.76,6.7,147,17.6
2,2013-06-07,11.5100,11.5200,11.0500,11.3050,12.6000,10.0260,11.31300,2,0,1,0,0,1,0.000,0.00,251,11.95,0.0,2738.89,2825.60,86.71,876.47,3.2,154,7.7


In [14]:
td1 = td.loc[1]
td1

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,cum_profit,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2009-09-18,8.4567,8.9045,8.4567,8.9045,8.7387,5.3119,7.02530,1,1,0,1,1,0,0.211,7123.6,800,11.95,0.0,7135.55,0.00,0.00,0.00,0.0,0,0.0
1,2009-10-30,9.0700,9.0700,8.2205,8.2205,9.3151,8.4567,8.88590,4,0,1,0,0,1,0.000,0.0,200,11.95,0.0,1783.89,1632.15,-151.74,-151.74,-8.5,42,-53.8
1,2009-10-30,9.0700,9.0700,8.2205,8.2205,9.3151,7.9027,8.60890,3,0,1,0,0,1,0.000,0.0,200,11.95,0.0,1783.89,1632.15,-151.74,-303.48,-8.5,42,-53.8
1,2010-06-04,10.5070,10.6280,10.3600,10.3730,11.6830,9.2100,10.44650,2,0,1,0,0,1,0.000,0.0,200,11.95,0.0,1783.89,2062.65,278.76,-24.72,15.6,259,22.7
1,2011-05-20,11.2200,11.2500,11.0600,11.0800,12.7000,9.6211,11.16055,1,0,1,0,0,1,0.000,0.0,200,11.95,0.0,1783.89,2204.05,420.16,395.44,23.6,609,13.5


In [15]:
# trade = td1.index[0]
# entry = td1.date.iloc[0]
# cost = td1.cost.iloc[0]
# exit = td1.date.iloc[-1]
# days = td1.days.iloc[-1]
# profit = td1.profit.sum()
# pct = (profit / cost) * 100
# annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
# td1_list = [trade, entry, cost, exit, days, profit, pct, annual]

In [16]:
# frame = pd.DataFrame(data=[td1_list], columns=['trade', 'entry', 'cost', 'exit', 'days', 'profit', 'pct', 'annual'])
# frame = frame.set_index('trade')
# frame = frame.round(2)
# frame

In [17]:
td.index.max()

6

In [18]:
for i in range(td.index.max()):
    print(i + 1)

1
2
3
4
5
6


In [19]:
for i in range(td.index.max()):
    df = td.loc[i + 1]
df

,date,open,high,low,close,upr,lwr,mid,sys,buy,sell,state,entry,exit,volatility,risk_amt,shares,charges,sduty,cost,value,profit,cum_profit,pct,days,annual
trade,,,,,,,,,,,,,,,,,,,,,,,,,,
6,2022-04-22,24.70,25.78,24.635,25.390,25.16,21.720,23.4400,1,1,0,1,1,0,0.077,19499.52,768,11.95,0.0,19511.47,0.00,0.00,8178.28,0.0,0,0.0
6,2022-05-13,25.53,25.53,24.905,24.905,26.50,23.825,25.1625,4,0,1,0,0,1,0.000,0.00,192,11.95,0.0,4877.87,4769.81,-108.06,8070.22,-2.2,21,-32.3


In [20]:
frame = pd.DataFrame(columns=['trade', 'entry', 'cost', 'exit', 'days', 'profit', 'pct', 'annual'])
frame

,trade,entry,cost,exit,days,profit,pct,annual


In [21]:
for i in [0]:
    df = td.loc[i + 1]
    trade = df.index[i]
    entry = df.date.iloc[i]
    cost = df.cost.iloc[i]
    exit = df.date.iloc[-1]
    days = df.days.iloc[-1]
    profit = df.profit.sum()
    pct = (profit / cost) * 100
    annual = ((1 + pct / 100) ** (365 / days) - 1) * 100
    trade_list = [trade, entry, cost, exit, days, profit, pct, annual]
trade_list

[1,
 Timestamp('2009-09-18 00:00:00'),
 7135.55,
 Timestamp('2011-05-20 00:00:00'),
 609,
 395.44000000000005,
 5.541829291365067,
 3.285506772772817]

In [ ]:
# Append result to frame!